In [9]:
import matplotlib
matplotlib.use('Agg')

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pickle
import random

from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization

In [10]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        if K.image_data_format() == 'channels_first':
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Conv2D(32, (3, 3), padding='same', use_bias=False, activation='relu', input_shape=inputShape))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.35))
        
        model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.35))
        
        model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.35))
        
        model.add(Conv2D(256, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(256, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.35))
        
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(classes, activation='softmax'))
        
        return model

In [11]:
EPOCHS = 100
INIT_LR = 5e-4
BS = 32
IMAGE_DIMS = (128, 128, 3)

data = []
labels = []

print('[INFO] loading images...')
imagePaths = sorted(list(paths.list_images('dataset')))
random.seed(2020)
random.shuffle(imagePaths)

[INFO] loading images...


In [12]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [13]:
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)
print('[INFO] data matrix: {:.2f}MB'.format(data.nbytes / (1024 * 1000.0)))

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

[INFO] data matrix: 712.70MB


In [14]:
aug = ImageDataGenerator(rotation_range=40,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [15]:
print('[INFO] compiling model...')
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0], depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()

[INFO] compiling model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 128, 32)      864       
_________________________________________________________________
batch_normalization_9 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 42, 42, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 42, 42, 64)        18432     
_________________________________________________________________
batch_normalization_10 (Batc (None, 42, 42, 64)        256       
_________________________________________________________________
conv2d_9 (Conv2D)           

In [16]:
##### print('[INFO] training network...')
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS,
    verbose=1
)

  ...
    to  
  ['...']
Train for 46 steps, validate on 372 samples
Epoch 1/100
46/46 [==============================] - 12s 252ms/step - loss: 1.7217 - accuracy: 0.5340 - val_loss: 1.6691 - val_accuracy: 0.2312
Epoch 2/100
46/46 [==============================] - 36s 783ms/step - loss: 1.1042 - accuracy: 0.6570 - val_loss: 2.5798 - val_accuracy: 0.2312
Epoch 3/100
46/46 [==============================] - 4s 86ms/step - loss: 0.9418 - accuracy: 0.7025 - val_loss: 1.8827 - val_accuracy: 0.2769
Epoch 4/100
46/46 [==============================] - 4s 86ms/step - loss: 0.8366 - accuracy: 0.7362 - val_loss: 3.5149 - val_accuracy: 0.2312
Epoch 5/100
46/46 [==============================] - 4s 86ms/step - loss: 0.8178 - accuracy: 0.7376 - val_loss: 3.2922 - val_accuracy: 0.2446
Epoch 6/100
46/46 [==============================] - 4s 86ms/step - loss: 0.6976 - accuracy: 0.7851 - val_loss: 1.5212 - val_accuracy: 0.3199
Epoch 7/100
46/46 [==============================] - 4s 86ms/step - loss: 0

In [17]:
from datetime import datetime
import pytz

In [18]:
tz = pytz.timezone('Asia/Ho_Chi_Minh')
today = datetime.now(tz)
today = today.strftime('%Y%m%d_%H%M')

In [19]:
model.save('model_{}.h5'.format(today))

f = open('label.pickle', 'wb')
f.write(pickle.dumps(lb))
f.close()

In [22]:
plt.style.use('ggplot')
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history['loss'], label='train_loss')
plt.plot(np.arange(0, N), H.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, N), H.history['accuracy'], label='train_accuracy')
plt.plot(np.arange(0, N), H.history['val_accuracy'], label='val_accuracy')
plt.title('Training Loss and Accuracy')
plt.ylim([0, 1.25])
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend(loc='upper left')
plt.savefig('plot_{}.png'.format(today))

In [23]:
print('Chart: plot_{}.png'.format(today))
print('Model: model_{}.h5'.format(today))

Chart: plot_20200506_0838.png
Model: model_20200506_0838.h5
